다양한 과일 사진이 업로드, 업로드된 사진을 클러스터로 분류하여 폴더 별로 저장
너무 많은 사진이 등록되어 저장 공간이 부족 -> 군집이나 분류에 영향을 끼치지 않으면서 사진의 용량을 줄일 수 있을까?

## 차원과 차원 축소
- 데이터가 가진 속성 = 특성
과일 사진의 경우 10000개의 픽셀이 있기 때문에 10000개의 특성이 있는 셈이다. 이런 특성을 '차원' 이라고도 부른다. 10000개의 특성 = 10000개의 차원. 이런 차원을 줄인다면 저장 공간 절약이 가능하다.

비지도 학습 작업 중 하나인 '차원 축소' 알고리즘을 이해하자.
특성이 많으면 선형 모델의 성능이 높아지고, 훈련 데이터에 쉽게 가대 적합된다는 것을 배웠다. 차원 축소는 데이터를 가장 잘 나타내는 일부 특성을 선택하여 데이터 크기를 줄이고 지도 학습 모델의 성능을 향상시킬 수 있는 방법.

또한 축소된 차원에서 원본차원으로 복원이 가능하다.

대표적인 차원 축소 알고리즘인 '주성분 분석'을 알아보자.

## 주성분 분석 소개
주성분 분석은 데이터에 있는 분산이 큰 방향을 찾는 것으로 볼 수 있다.
- 분산 = 데이터가 널리 퍼져있는 정도
널려있는 데이터에서 분산이 가장 큰 방향? 데이터들의 중심을 관통하는 직선

이 벡터를 '주성분'이라고 한다. 이 주성분 벡터의 원소 개수 = 원본 데이터셋에 있는 특성 개수와 같다. 

원본 샘플 데이터를 주성분에 직각으로 투영하면 주성분 위의 점 데이터로 만들 수 있다.

주성분은 원본 차원과 같고, 주성분으로 바꾼 데이터는 차원이 줄어든다. 주성분이 가장 분산이 큰 방향이기 때문에 주성분에 투영하여 바꾼 데이터는 원본이 가지고 있는 특성을 가장 잘 나타내고 있을 것이다.

첫번째 주성분을 찾은 후 이 벡터에 수직이고 분산이 가장 큰 다음 방향을 찾는다.

## PCA 클래스

In [ ]:
!wget https://bit.ly/fruits_300 -O fruits_300.npy
import numpy as np
fruits = np.load('fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

사이킷런은 sklearn.decomposition 모듈 아래 PCA 클래스로 주성분 분석 알고리즘을 제공한다. PCA 클래스의 객체를 만들 때 n_components 매개변수에 주성분의 개수를 지정해야 한다. k-평균과 마찬가지로 비지도학습이기 때문에 fit() 메서드에 타깃값은 제공X

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 50)
pca.fit(fruits_2d)

In [ ]:
print(pca.components_.shape)

n_components=50으로 설정했기 때문에 첫번째 차원이 50. 즉 50개의 주성분을 찾은 것이다. 두번째 차원은 원본 데이터의 특성 개수와 같은 10000

In [ ]:
import matplotlib.pyplot as plt
def draw_fruits(arr, ratio = 1):
    n = len(arr) # arr = 샘플의 개수
    # 한 줄에 10개씩 이미지를 그림, 샘플의 개수를 10으로 나누어 전체 행 개수 계산
    rows = int(np.ceil(n/10))
    # 행이 1개이면 열의 개수는 샘플의 개수, 그렇지않으면 10개
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols, figsize = (cols*ratio, rows*ratio), squeeze=False)
    for i in range(rows):
        for j in range(cols):
            if i*10 + j < n: # n개까지만 그린다
                axs[i,j].imshow(arr[i*10 + j], cmap='gray_r')
            axs[i,j].axis('off')
    plt.show()

draw_fruits를 통해 주성분을 이미지처럼 출력 가능

In [ ]:
draw_fruits(pca.components_.reshape(-1,100,100))

이 주성분은 원본 데이터에서 가장 분산이 큰 방향을 순서대로 나타낸 것.
주성분을 찾았으니 원본 데이터를 주성분에 투영하여 특성의 개수를 10000개에서 50개로 줄임
PCA의 transform() 메서드를 사용

In [ ]:
print(fruits_2d.shape)

In [ ]:
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape)

fruits_2d는 (300, 10000)이였던 크기의 배열. 10000개의 픽셀(특성)을 가진 300개의 이미지. 50개의 주성분을 찾은 PCA 모델을 사용해 (300,50) 크기의 배열로 변환.

원상 복구도 가능하다.

## 원본 데이터 재구성
10000개의 특성을 50개로 줄이면 손실이 발생한다. 최대한 분산이 큰 방향으로 데이터를 투영했기 때문에 원본 데이터를 상당 부분 재구성 가능.

PCA 클래스는 이를 위해 inverse_transform() 메서드를 제공해 50개 차원으로 축소한 fruits_pca 데이터를 10000개의 특성으로 복원한다.

In [ ]:
fruits_inverse = pca.inverse_transform(fruits_pca)
print(fruits_inverse.shape)

10000개의 특성으로 복원된 것을 볼 수 있다. 복원한 데이터를 100x100 크기로 바꾸어 100개씩 나누어 출력을 해본다.


In [ ]:
fruits_reconstruct = fruits_inverse.reshape(-1, 100, 100)
for start in [0,100,200]:
    draw_fruits(fruits_reconstruct[start:start+100])
    print("\n")

모든 데이터가 복원이 되었지만 번진 것을 볼 수 있다. 하지만 괜찮은 결과이다.

50개의 특성을 10000개로 늘린 것을 감안하면 괜찮은 것이다. 이 50개의 특성이 분산을 가장 잘 보존하도록 변환된 것이기 때문.

## 설명된 분산
주성분이 원본 데이터의 분산을 얼마나 잘 나타내는지 기록한 값을 '설명된 분산'이라고 한다. PCA 클래스의 explained_variance_ratio_에 각 ㅅ주성분의 설명된 분산 비율이 기록. 첫번째 설명된 분산이 가장 크다. 이 분산 비율을 모두 더하면 50개의 주성분으로 표현하고 있는 총 분산 비율을 얻을 수 있다.(설명된 분산은 주성분이 원본 데이터의 분산을 얼마나 잘 나타내는지 기록한 값)


In [ ]:
print(np.sum(pca.explained_variance_ratio_))

92퍼센트가 넘는 분산을 유지. 설명된 분산의 비율을 그래프로 그리면 적절한 주성분의 개수를 찾는데 도움이 된다.

In [ ]:
plt.plot(pca.explained_variance_ratio_)

그래프를 보면 처음 10개의 주성분이 대부분의 분산을 표현.

이번에는 PCA로 차원 축소된 데이터를 사용하여 지도 학습 모델을 훈련.

## 다른 알고리즘과 함께 사용하기
원본 과일 사진 데이터와 PCA로 축소한 데이터를 지도 학습에 적용해보고 어떤 차이가 있는지 알아보자. 로지스틱 회귀 모델을 사용

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

지도 학습이기 때문에 타깃값을 준다. 사과 0, 파인애플1, 바나나2


In [ ]:
target = np.array([0]*100 + [1]*100 + [2]*100)
target

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(lr, fruits_2d,target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

교차 검증 점수는 매우 높고, 특성이 많기 때문에 과대 적합이 되는 모델이 되기가 쉽다. cross_validate() 함수가 반환하는 딕셔너리에는 fit_time 항목에 각 교차 검증 폴드의 훈련 시간이 기록. 여기서는 1.4초 정도가 걸림

다음 PCA로 축소한 fruits_pca를 사용했을 때 비교

In [ ]:
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

50개의 특성만 사용했는데 정확도가 100프로, 훈련 시간은 0.02초로 감소

PCA클래스를 사용할 때 n_components 매개변수에 주 성분의 개수를 지정. 이 대신 원하는 설명된 분산의 비율을 입력할 수도 있다. PCA 클래스는 지정된 비율에 도달할 때까지 자동으로 주성분을 찾는다. 설명된 분산의 50%에 달하는 주성분을 찾도록 PCA 모델 만든다.

In [ ]:
pca = PCA(n_components=0.5)
pca.fit(fruits_2d)

주성분의 개수 대신 0~1사이의 숫자를 입력하면 된다.

In [ ]:
print(pca.n_components_)

2개의 특성만으로 원본 데이터에 있는 분산의 50%를 표현할 수 있다.

In [ ]:
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape)

2개의 특성으로만 교차 검증 결과를 함


In [ ]:
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

로지스틱 회귀 모델이 완전히 수렴하지 않았기 때문에 반복횟수를 증가시키라는 오류가 뜨지만 무시해도 괜찮다. 

2개의 특성으로만 99%의 정확도를 달성.

이번에는 차원 축소된 데이터를 사용해 k-평균 알고리즘으로 클러스터를 찾아봄

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state=42)
km.fit(fruits_pca)
print(np.unique(km.labels_, return_counts=True))

fruits_pca로 찾은 클러스터 91, 99, 110개의 샘플 가지고있음. 이는 원본 데이터와 비슷한 결과를 나타냄.

In [ ]:
for label in range(0,3):
    draw_fruits(fruits[km.labels_ == label])
    print("\n")

훈련 데이터의 차원을 줄이면 시각화를 할 수 있다는 장점이 있다. 3개 이하로 차원을 줄이면 화면에 출력하기 쉽다.

In [ ]:
for label in range(0,3):
    data = fruits_pca[km.labels_ == label]
    plt.scatter(data[:,0], data[:,1])

plt.legend(['apple','pineapple','banana'])
plt.show()

이 산점도를 통해 사과와 바나나의 경계가 가깝게 붙어서 혼동을 일으키기 쉽다는 것을 알 수 있다.

시각화를 통해 이러한 면을 알 수 있다.